# Geoprocessing in Python #

The Federal State of Nordrhein-Westfalen (NRW, North Rhine-Westphalia) provides Digital Terrain Models (DTM) gained from Airborne Laser Scanning (ALS). One of the derived remote sensing products is an interpolated DTM on a regular grid with 1m² horizontal (x-y-plane) grid size (1m x 1m). 

The DTM data is dissected in regular square tiles with 2000 x 2000 grid cells each corresponding to 2000 m x 2000 m metric area. These tiles are organized according to the municipalities in NRW. All tiles touching a municipality are pooled in archive and compressed (zipped).

But the chosen file format to store the tiles is strange. The original file format of each tile is a formatted text file with three columns for the x-y-z coordinates denoted as XYZ file format. Each row in the text file represents one grid point in space. Each cell is represented by its full x-y-coordinates in the chosen coordinate reference system (EPSG:4647) together with the interesting information: the elevation z. 

Since each tile is a square grid with 2000 x 2000 cells it is regular and it would be enough e.g. to store the lower left corner coordinates together with the organisation of the grid cells (e.g. 2000 rows, 2000 columns, rows stored firstly) and then the pure z-coordinates sequentially. The x-y-coordinates of each grid cell could be determined by counting the z-values in the file: z-value at position 0 is at lower left corner, z-value at position 3'999'999 is at upper right corner, z-value at 1999 is at lower right corner (depending on whether data is stored row-wise or column-wise). 

Sometimes the upper right corner is the reference point. It just has to be described how data is organized.

geoTiff is a file format suited for handling regular geo-referenced grids.

The following activity deals with the transfromation from xyz-format to geoTiff.

But first the the bounding boxes of the available DTM tiles have to be determined to see which area they cover and whether they lie in the actual region of interest. 




### Prerequisites

* Install the Anaconda3 Python distribution.
* Open the Anaconda Prompt (bash-like terminal) and install the package gdal: `conda install gdal`

In [1]:
import os, sys
from osgeo import osr, ogr, gdal

In [6]:
#help(osr)
#help(ogr)
#help(gdal)

### Relative Path to Directory with the DTM XYZ-Files

In [7]:
# get the current working directory
os.getcwd()

'C:\\Users\\rb\\Documents\\HSRW\\Teaching\\MIE_GeoInfo\\MIE_GeoInfo_WS2020\\REPO\\MIE_2.02_GeoInfo_WS2020\\gi0301_ALS_DTM_NRW'

In [8]:
#where the xyz files are
dtm_xyz_dir = "../data/DTM_Xanten/dgm1_05170052_Xanten_EPSG4647_XYZ/"
dest_dir = "../data/derived/DTM_Xanten/"
dest_shapefile_name = "dgm1_Xanten_BB_coverage.shp" # shape file with bounding boxes, DGM: Digitales Geländemodell (de) <-> Digital Terrain Model (en)

dest_geotiff_dir = "../data/derived/DTM_Xanten/geoTiff/" ## directory of geoTiff files 

os.makedirs(dest_dir, exist_ok = True)
os.makedirs(dest_geotiff_dir, exist_ok = True)

In [ ]:
os.listdir(dtm_xyz_dir)

### Create a list with all xyz-file names

In [ ]:
fnamelist = []
for fname in os.listdir(dtm_xyz_dir):
    if fname.endswith(".xyz"):
        fnamelist.append(fname)

fnamelist

### Define a function which evaluates first and last row to get the bounding box

In [ ]:
def getBB (fname, pixelsize):
    with open(fname, 'rb') as fh:
        first = next(fh)
        offs = -100
        while True:
            fh.seek(offs, 2)
            lines = fh.readlines()
            if len(lines)>1:
                last = lines[-1]
                break
            offs *= 2
        print("first: ", first)
        print ("last: ", last)

        [left, bottom, z_ws] = list(map(float, first.split()))
        [right, top, z_en] = list(map(float, last.split()))
        left, right, bottom, top = left - pixelsize/2., right + pixelsize/2., bottom - pixelsize/2., top + pixelsize/2.

        return [left, right, bottom, top] 

### Create a shape file the rectangular bounding boxes of the xyz files

In [ ]:
# get the driver
driver = ogr.GetDriverByName('ESRI Shapefile')
# create a new data source and layer

#pathname = myDestDataDir + r'/dgm1_Xanten_BB_coverage.shp'

pathfilename = dest_dir + dest_shapefile_name

print("Output File: ", pathfilename)

if os.path.exists(pathfilename):
    driver.DeleteDataSource(pathfilename)

ds = driver.CreateDataSource(pathfilename)

if ds is None:
    print('Could not create file')
    sys.exit(1)


#~ src_srs = txt2srs('EPSG:4647')
#src_srs= txt2srs('+proj=tmerc +lat_0=0 +lon_0=9 +k=0.9996 +x_0=32500000 +y_0=0 +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs')    

epsg = 4647
srs1 = osr.SpatialReference()
srs1.ImportFromEPSG(epsg)

#layer_name = 'test'
#layer_name = layer_name.encode('utf-8')
#layer = ds.CreateLayer(layer_name, srs, ogr.wkbPolygon)

layer = ds.CreateLayer('test', geom_type=ogr.wkbPolygon, srs=srs1)
#layer = ds.CreateLayer('test', geom_type=ogr.wkbPolygon)

fieldDefnID = ogr.FieldDefn('id', ogr.OFTInteger)
fieldDefnFName = ogr.FieldDefn('fname', ogr.OFTString)

res = layer.CreateField(fieldDefnID)
res =layer.CreateField(fieldDefnFName)


# get the FeatureDefn for the output layer
featureDefn = layer.GetLayerDefn()

# ID
i=0

pixelsize = 1

for fname in fnamelist:
    [x0, x1, y0, y1] = getBB(dtm_xyz_dir+fname, pixelsize)
    #print(x0, x1, y0, y1)
    ring = ogr.Geometry(ogr.wkbLinearRing)
    ring.AddPoint(x0,y0)
    ring.AddPoint(x1,y0)
    ring.AddPoint(x1,y1)
    ring.AddPoint(x0,y1)
    ring.AddPoint(x0,y0)
    #ring.CloseRings() # or ring.AddPoint(0,0)
    
    polygon = ogr.Geometry(ogr.wkbPolygon)
    polygon.AddGeometry(ring)
    
    feature = ogr.Feature(featureDefn)
    feature.SetGeometry(polygon)

    i+=1
    feature.SetField('id', i)
    feature.SetField('fname', fname)
    # add the feature to the output layer
    layer.CreateFeature(feature)
    
    #ring.Destroy()
    #polygon.Destroy()
    #feature.Destroy()

ds.Destroy()


### Add the four filenames touching the GPS path in Hees and on Fürstenberg to the list.

In [ ]:
# The files the GPS track is intersecting with
hees_dtm_list=["dgm1_...xyz","dgm1_....xyz","dgm1_....xyz","dgm1_....xyz"]

The following example is derived from here:  
https://gis.stackexchange.com/questions/58517/python-gdal-save-array-as-raster-with-projection-from-other-fil  
and here:
https://gis.stackexchange.com/questions/177061/ascii-file-with-latitude-longitude-and-data-to-geotiff-using-python


In [ ]:
def CreateGeoTiff(Name, Array, driver, 
                  xnum, ynum, GeoT):
    DataType = gdal.GDT_Float32
    NewFileName = Name+'.tif'
    
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4647)
    # Set up the dataset
    DataSet = driver.Create( NewFileName, xnum, ynum, 1, DataType )
            # the '1' is for band 1.
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection(srs.ExportToWkt())

    # Write the array
    DataSet.GetRasterBand(1).WriteArray( Array )
    return NewFileName

In [ ]:

for source in hees_dtm_list:
    base=os.path.basename(source)
    name = os.path.splitext(base)[0]
    print(name)

In [ ]:
hees_dtm_list

In [ ]:
import pandas as pd
from os.path import basename

xnum, ynum = 2000, 2000
xsize, ysize = 1,1
driver = gdal.GetDriverByName('GTiff')


for source in hees_dtm_list:
    print("read " + source)
    df=pd.read_csv(dtm_xyz_dir + source,header=None,delim_whitespace=True,names=["x","y","z"])
    xmin, xmax = min(df.x), max(df.x)
    ymin, ymax = min(df.y), max(df.y)
    zmin, zmax = min(df.z), max(df.z)
    GeoTNew=[xmin, xsize, 0, ymax, 0, -ysize] # WHY IS IT LIKE THIS ?????? WHAT DOES IT DO ?????
    
    z=(df.z).values.reshape(ynum,xnum)
    z=np.rot90(z) # If we do not rotate the 2000x2000 pixel array the final result is rotated!

    base=os.path.basename(source)
    name = os.path.splitext(base)[0]
    
    print("write " + name + ".tif")    
    NewFileName = CreateGeoTiff(dest_geotiff_dir + name, z, driver, xnum, ynum, GeoTNew)
    